## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [2]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score


import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [3]:
# Cells for you to work and document as necessary - 
# definitely feel free to add more cells as you need

In [4]:
train_data.shape[0]

84534

In [5]:
train_data.V7.value_counts()

2    59317
1    25217
Name: V7, dtype: int64

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84534 entries, 0 to 84533
Data columns (total 10 columns):
ID           84534 non-null int64
Promotion    84534 non-null object
purchase     84534 non-null int64
V1           84534 non-null int64
V2           84534 non-null float64
V3           84534 non-null float64
V4           84534 non-null int64
V5           84534 non-null int64
V6           84534 non-null int64
V7           84534 non-null int64
dtypes: float64(2), int64(7), object(1)
memory usage: 6.4+ MB


In [7]:
y = train_data.purchase
x = train_data.drop(columns = ['purchase' ,'ID'],axis=1)
dummy_response = pd.get_dummies(train_data['Promotion'])['Yes']


In [8]:
df = train_data.copy()
df = pd.concat([df, dummy_response] ,axis = 1)
df = df.drop(['Promotion','ID'], axis = 1)
df.head()

,purchase,V1,V2,V3,V4,V5,V6,V7,Yes
0,0,2,30.443518,-1.165083,1,1,3,2,0
1,0,3,32.159350,-0.645617,2,3,2,2,0
2,0,2,30.431659,0.133583,1,1,4,2,0
3,0,0,26.588914,-0.212728,2,1,4,2,0
4,0,3,28.044332,-0.385883,1,1,2,2,1


### The test below shows the baseline value for if we were to give everyone a promotion. Let's see what IRR and NIR this gives us


In [9]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    length = df.shape[0]
    promotion = []
    for i in range(length):
        promotion.append('Yes')
    promotion = np.array(promotion)
    return promotion

In [10]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0096.

Your nir with this strategy is -1132.20.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.0095931582782501079, -1132.1999999999998)

### Let's try and improve this NIR and IRR socre by training a model on just the people that got the promotion and made a purchase (MODEL 1)

> Let's look at user's who were given a promotion and also bought the $10 item and use this to create our model

In [11]:
df = df.rename(columns={"Yes": "Promotion"})
df.head()

,purchase,V1,V2,V3,V4,V5,V6,V7,Promotion
0,0,2,30.443518,-1.165083,1,1,3,2,0
1,0,3,32.159350,-0.645617,2,3,2,2,0
2,0,2,30.431659,0.133583,1,1,4,2,0
3,0,0,26.588914,-0.212728,2,1,4,2,0
4,0,3,28.044332,-0.385883,1,1,2,2,1


In [12]:
# Create a new column that keeps track of if the user was given a 
# promotion and also made a purchase

success = []
for i in range(df.shape[0]):
    if((df['Promotion'][i] == 1) and (df['purchase'][i] == 1)):
        success.append(1)
    else:
        success.append(0)
df['success'] = success
df.head()

,purchase,V1,V2,V3,V4,V5,V6,V7,Promotion,success
0,0,2,30.443518,-1.165083,1,1,3,2,0,0
1,0,3,32.159350,-0.645617,2,3,2,2,0,0
2,0,2,30.431659,0.133583,1,1,4,2,0,0
3,0,0,26.588914,-0.212728,2,1,4,2,0,0
4,0,3,28.044332,-0.385883,1,1,2,2,1,0


In [13]:
y = df['success']
X = df.drop(['success','Promotion','purchase'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [14]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.98089400293938411

In [15]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''

    y_pred = clf.predict(df)
    
    promotion = []
    for i in range(df.shape[0]):
        if y_pred[i] == 1:
            promotion.append('Yes')
        else:
            promotion.append('No')
    promotion = np.array(promotion)
    
    
    return promotion

In [16]:
test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0234.

Your nir with this strategy is 23.40.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.023441090853378276, 23.399999999999999)

### Our model did a lot better this time, especially in the IRR front! However, we only looked at purchases made by people who were given the promotion. We can also try and generalize this to make it for anybody who made a purchase
> Simple as making success for anyone, not just those given a promotion

In [17]:
df2 = df.copy()
df2 = df2.drop('success', axis = 1)
df2.head()

,purchase,V1,V2,V3,V4,V5,V6,V7,Promotion
0,0,2,30.443518,-1.165083,1,1,3,2,0
1,0,3,32.159350,-0.645617,2,3,2,2,0
2,0,2,30.431659,0.133583,1,1,4,2,0
3,0,0,26.588914,-0.212728,2,1,4,2,0
4,0,3,28.044332,-0.385883,1,1,2,2,1


In [18]:
y = df2['purchase']
X = df2.drop(['Promotion','purchase'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.97078538910994017

In [19]:
test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0127.

Your nir with this strategy is -8.30.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.012730121425773599, -8.2999999999999972)

### Did a lot worse (sort of as expected). Let's see if we identify each type of group how it goes

In [20]:
df.head()

,purchase,V1,V2,V3,V4,V5,V6,V7,Promotion,success
0,0,2,30.443518,-1.165083,1,1,3,2,0,0
1,0,3,32.159350,-0.645617,2,3,2,2,0,0
2,0,2,30.431659,0.133583,1,1,4,2,0,0
3,0,0,26.588914,-0.212728,2,1,4,2,0,0
4,0,3,28.044332,-0.385883,1,1,2,2,1,0


In [29]:
df3 = df.copy()
success = []
for i in range(df.shape[0]):
    if((df['Promotion'][i] == 1) and (df['purchase'][i] == 1)):
        success.append(1)
    elif((df['Promotion'][i] == 1) and (df['purchase'][i] == 0)):
        success.append(-1)
    else:
        success.append(0)
df3['success'] = success
df3.head()

,purchase,V1,V2,V3,V4,V5,V6,V7,Promotion,success
0,0,2,30.443518,-1.165083,1,1,3,2,0,0
1,0,3,32.159350,-0.645617,2,3,2,2,0,0
2,0,2,30.431659,0.133583,1,1,4,2,0,0
3,0,0,26.588914,-0.212728,2,1,4,2,0,0
4,0,3,28.044332,-0.385883,1,1,2,2,1,-1


In [30]:
y = df3['success']
X = df3.drop(['Promotion','purchase', 'success'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

clf3 = DecisionTreeClassifier()
clf3.fit(X_train, y_train)
y_pred = clf3.predict(X_test)
accuracy_score(y_test, y_pred)

0.49227515503459152

In [31]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''

    y_pred = clf3.predict(df)
    
    promotion = []
    for i in range(df.shape[0]):
        if y_pred[i] == 1:
            promotion.append('Yes')
        else:
            promotion.append('No')
    promotion = np.array(promotion)
    
    
    return promotion

In [32]:
test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is -0.0047.

Your nir with this strategy is -43.45.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(-0.0047300202715154487, -43.449999999999996)